# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from drivers.sandbox import Sandbox
from drivers.generator import Generator
from drivers.optimiser import Optimiser

from inference_toolbox.domain import Domain
from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Define how the data is to be generated and what default inputs should be used

In [2]:
data_params = {
    'data_type': 'dummy',
    'data_path': 'data',
    'sigma': 1,
    'model_select': 'log_gpm_alt_norm',
    'noise_dist': 'gaussian',
    'model': {
        'model_params':{
            'H': 10
        },
        'inference_params':{
            'I_y': 0.1,
            'I_z': 0.1,
            'Q': 3e13
        },
    },
    'domain': {
        'domain_select': 'cone_from_source_z_limited', 
        'resolution': 20,
        'domain_params':{
            'r': 100,
            'theta': np.pi/8,
            'source': [0,0,10]}
    },
    'output_header': 'Concentration'
}

default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'I_z': Parameter('I_z', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'Q': Parameter('Q', prior_select = 'gamma', default_value=3e13).add_prior_param('mu', 3e13).add_prior_param('sigma',1e13),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'gamma', default_value=1).add_prior_param('mu', 1).add_prior_param('sigma',1)
        })
    }),
    'model':Model('log_gpm_alt_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

results_name = 'simulated_data'

## Sandbox ##

Create an instance with the inputted default parameters and visualise it in different ways

In [3]:
sandbox = Sandbox(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 100)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

TypeError: can't multiply sequence by non-int of type 'float'

## Generator ##

Generate different instances by varying the hyperparameters with respect to their defaults

In [ ]:
generator = Generator(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

Vary one hyperparameter

In [ ]:
analysis_iterations = {
    'parameters':
    [
        'likelihood_sigma', 
    ],
    'values':
    [
        np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4])
    ]
}

for i in range(len(analysis_iterations['parameters'])):
    parameter = analysis_iterations['parameters'][i]
    print('Working on varying ' + parameter + '...')
    values = analysis_iterations['values'][i]
    inputs = generator.vary_one_parameter(parameter, values, xscale='linear', plot=True)

Working on varying likelihood_sigma...
Generating instance 1...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5913.85it/s, 3 steps of size 6.66e-01. acc. prob=0.92]


Generating instance 2...


warmup:   4%|▍         | 561/12500 [00:01<00:28, 420.56it/s, 1 steps of size 3.90e-01. acc. prob=0.79]


KeyboardInterrupt: 

Vary two hyperparameters

In [ ]:
analysis_iterations = {
    'parameters_1':
    [
        'I_y_sigma', 
    ],
    'parameters_2':
    [
        'likelihood_sigma', 
    ],
    'values_1':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
    ],
    'values_2':
    [
        np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
    ]
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='log', scale_2='linear', plot=True)

Working on varying I_y_sigma...
Generating instance 1...


sample: 100%|██████████| 12500/12500 [00:29<00:00, 416.91it/s, 511 steps of size 3.02e-07. acc. prob=0.80] 


Generating instance 2...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 4715.87it/s, 3 steps of size 3.65e-01. acc. prob=0.93] 


Generating instance 3...


...

sample: 100%|██████████| 12500/12500 [00:02<00:00, 5666.03it/s, 3 steps of size 8.43e-01. acc. prob=0.89]


Generating instance 99...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5592.02it/s, 3 steps of size 7.12e-01. acc. prob=0.93]


Generating instance 100...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5647.36it/s, 3 steps of size 7.95e-01. acc. prob=0.91]


## Optimiser ##

Optimise the default construction using Bayesian Optimisation

In [ ]:
optimising_parameters = {
                    'I_y_mu': [1e-2, 10],
                    'I_y_sigma': [1e-2, 10],
                    'I_z_mu': [1e-2, 10],
                    'I_z_sigma': [1e-2, 10],
                    'Q_mu': [1e9, 1e18],
                    'Q_sigma': [1e9, 1e18],
                }

optimiser = Optimiser(results_name=results_name, 
                  default_params=default_params, 
                  data_params=data_params)

study = optimiser.run(n_trials=10, optimiser_name='optimiser_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)

optimiser.run_best_params(study)

[I 2023-07-06 00:21:47,140] A new study created in memory with name: no-name-cc165d2b-6bdf-4d36-b8c5-833100a69fa9


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5038.78it/s, 7 steps of size 1.88e-01. acc. prob=0.93] 
[I 2023-07-06 00:21:58,484] Trial 0 finished with value: 116.59412384033203 and parameters: {'I_y_mu': 7.242819636870754, 'I_y_sigma': 5.38475701400792, 'I_z_mu': 2.7184932409094564, 'I_z_sigma': 6.154708994918618, 'Q_mu': 9.417718564821115e+17, 'Q_sigma': 4.1170024922262496e+17}. Best is trial 0 with value: 116.59412384033203.
sample: 100%|██████████| 12500/12500 [00:02<00:00, 5040.89it/s, 3 steps of size 1.49e-01. acc. prob=0.92] 
[I 2023-07-06 00:22:09,968] Trial 1 finished with value: 115.3094482421875 and parameters: {'I_y_mu': 9.997879256923916, 'I_y_sigma': 2.9375794646617095, 'I_z_mu': 2.9383912510108017, 'I_z_sigma': 7.178588319400818, 'Q_mu': 7.95869135102434e+17, 'Q_sigma': 7.305416591379677e+17}. Best is trial 1 with value: 115.3094482421875.
sample: 100%|██████████| 12500/12500 [00:02<00:00, 5193.28it/s, 7 steps of size 2.65e-01. acc. prob=0.94] 
[I 2023-07-06 00:22:2

Study statistics: 
 Number of finished trials:  10
 Number of pruned trials:  0
 Number of complete trials:  10
Best trial:
 Value:  113.13556671142578
Params: 
    I_y_mu: 3.768053988772884
    I_y_sigma: 5.963543000282927
    I_z_mu: 8.52529614871596
    I_z_sigma: 8.402146419052892
    Q_mu: 3.388184125011668e+17
    Q_sigma: 5.6345523163107e+17


sample: 100%|██████████| 12500/12500 [00:02<00:00, 5541.74it/s, 3 steps of size 5.88e-01. acc. prob=0.91]
